# BigMart Sales Prediction
Data scientists at BigMart have collected 2013 sales data for 1559 products across 10 stores in different cities.        
Also, certain attributes of each product and store have been defined.              
The aim of this data science project is to build a **predictive model and find out the sales of each product at a particular store.**

# Imports

In [2]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns

# Get Data and Info

In [3]:
data = pd.read_csv('C:\\Users\\kruth\\OneDrive\\Desktop\\DS Lab Case study datasets\\BigMartSales\\Train.csv')
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [5]:
print("Target Variable: Item_Outlet_Sales")
data.iloc[:,-1]

Target Variable: Item_Outlet_Sales


0       3735.1380
1        443.4228
2       2097.2700
3        732.3800
4        994.7052
          ...    
8518    2778.3834
8519     549.2850
8520    1193.1136
8521    1845.5976
8522     765.6700
Name: Item_Outlet_Sales, Length: 8523, dtype: float64

## Observations
From the above information, we can note that:
1. There are 11 features and 1 target variable (Item_Outlet_Sales).
2. This is a Supervised Machine Learning problem with Regression.
3. There are 7 categorical features and 4 numerical features.
4. The data is not scaled.
5. There are missing values in the data.
6. There might be some outliers in the data.

# Data Preprocessing

## Identify Categorical and Numerical Features

In [7]:
cat_cols = data.select_dtypes(include='object').columns
num_cols = data.select_dtypes(exclude='object').columns
print("Categorical Columns: \n",cat_cols)
print("Numerical Columns: \n",num_cols)

Categorical Columns: 
 Index(['Item_Identifier', 'Item_Fat_Content', 'Item_Type', 'Outlet_Identifier',
       'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type'],
      dtype='object')
Numerical Columns: 
 Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year', 'Item_Outlet_Sales'],
      dtype='object')


In [8]:
# separate target variable
y = data.iloc[:,-1]

In [11]:
for col in cat_cols:
    print(data[col].value_counts(),"\n") 

Item_Identifier
FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: count, Length: 1559, dtype: int64 

Item_Fat_Content
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: count, dtype: int64 

Item_Type
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: count, dtype: int64 

Outlet_Identifier
OUT027    935
OUT013    932
OUT049    930
OUT046    930
OUT035    930
OUT045    929
OUT018    928
OUT017    926
OUT010    555
OUT019    528
Name: cou

Observations for categorical features:
1. Item_Identifier - numerical values exist but as strings
2. Item_Fat_Content - 5 categories, Ordinal
3. Item_Type - many categories, Nominal
4. Outlet_Identifier - many categories, Nominal
5. Outlet_Size - 3 categories, Ordinal
6. Outlet_Location_Type - 3 categories, Ordinal
7. Outlet_Type - 4 categories, Ordinal


In [13]:
data[num_cols].head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
0,9.30,0.016047,249.8092,1999,3735.1380
1,5.92,0.019278,48.2692,2009,443.4228
2,17.50,0.016760,141.6180,1999,2097.2700
3,19.20,0.000000,182.0950,1998,732.3800
4,8.93,0.000000,53.8614,1987,994.7052


These numerical features need to be scaled

## Duplicates

In [14]:
data.duplicated().sum()

0

No duplicates found

## Missing Values

In [15]:
data.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

Item_weight is numerical and Outlet_Size is categorical. Both have missing values.

## Transform all features

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer # missing values
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import TransformerMixin, BaseEstimator

# define a tranformer for Item_Identifier - numerical values exist but as strings
class CatToFloat(TransformerMixin, BaseEstimator):

    # X is the col which has numerical values exist but as strings
    # it should be numpy array or series of 1D
    def __init__(self):
        pass

    def fit(self, X, y = None):
        return self
    
    def transform(self, X):

        transformed = []
        # get the numeric part of each instance and append it to transformed
        for instance in X:
            # get the numeric part
            # store digit in list if ch.isdigit() is returned True
            # join the digits found in particular instance
            numeric_part = "".join([ch for ch in instance if ch.isdigit()])
            
            





